# Epoch check

In this notebook, we count the number of epochs for each brain state.

**Possible brain states:**
 
0 : Wake 

1 : REM

2 : N-REM

3 : Artifact/Harmonic

4 : Seizure

5 : Noise

6 : Packet Loss

### Import packages 

In [50]:
import pandas as pd
import numpy as np
import os
import pickle
from collections import defaultdict


### Load files

In [51]:
clean_br_path = "/exports/eddie/scratch/s2864332/SYNGAP_Rat_Data/Preprocessed/clean_brain_states"
OUTPUT_CSV = "/home/s2864332/MySYNGAP/MySYNGAP/DiagnoseSYNGAP/Scripts/Analyse/Rat_brain_state_epoch_counts_nikoo.csv"


In [52]:
STATE_NAMES = {
    0: "wake",
    1: "rem",
    2: "nrem",
    3: "artifact",
    4: "seizure",
    5: "noise",
    6: "packet_loss",
    7: "total",
    8: "total_clean",
    9: "total_noisy"
}

In [53]:
# container: one dict per animal
animal_data = {}

for fname in os.listdir(clean_br_path):
    if not fname.endswith(".pkl"):
        continue

    animal_id = fname.split("_")[0]  
    pkl_path = os.path.join(clean_br_path, fname)

    with open(pkl_path, "rb") as f:
        br_dict = pickle.load(f)

    brainstates = br_dict["brainstate"]

    if animal_id not in animal_data:
        animal_data[animal_id] = {
            "Animal_ID": animal_id,
            "wake_epochs": 0,
            "rem_epochs": 0,
            "nrem_epochs": 0,
            "artifact_epochs": 0,
            "seizure_epochs": 0,
            "noise_epochs": 0,
            "packet_loss_epochs": 0,
            "total_epochs": 0,
            "clean_epochs": 0,
            "noisy_epochs": 0
        }

    for state in brainstates:
        # count total epochs
        animal_data[animal_id]["total_epochs"] += 1

        # count per-state
        state_name = STATE_NAMES.get(state)
        if state_name is not None:
            animal_data[animal_id][f"{state_name}_epochs"] += 1

        # clean vs noisy
        if state in [0, 1, 2]:
            animal_data[animal_id]["clean_epochs"] += 1
        elif state in [3, 4, 5, 6]:
            animal_data[animal_id]["noisy_epochs"] += 1

# build dataframe
df = pd.DataFrame(list(animal_data.values()))
df = df.sort_values("Animal_ID").reset_index(drop=True)

df.to_csv(OUTPUT_CSV, index=False)
print(f"Saved results to {OUTPUT_CSV}")

Saved results to /home/s2864332/MySYNGAP/MySYNGAP/DiagnoseSYNGAP/Scripts/Analyse/Ratbrain_state_epoch_counts.csv


In [54]:
print(df)

   Animal_ID  wake_epochs  rem_epochs  nrem_epochs  artifact_epochs  \
0      S7063           35       12105          590            10241   
1      S7064            7       10069         1082            18616   
2      S7068          123           0           28                0   
3      S7069           75       11785          888            18641   
4      S7070            5       12037         2113            19316   
5      S7071          605       12589         1346            16431   
6      S7072            7       11309          634            19331   
7      S7074         1366          14           79                0   
8      S7075         1217           4           48                0   
9      S7076          702          11           84                0   
10     S7083            6        5837          154            13308   
11     S7086           74       10995          218            15241   
12     S7088          486           5           33                0   
13    